In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] =15,9
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('sensity_events.csv')

In [3]:
df.head()

,uuid,asset_id,start_time,end_time,session_type,latitude,longitude,geometry
0,SENSITY-kc-23-40D98DC1-41F8-40EE-BBB8-E44DE41C...,SENSITY-kc-23,2018-08-10 23:33:26.081,2018-08-10 23:33:55.9371,car,39.109786,-94.583482,"{""type"":""Polygon"",""coordinates"":[[[-94.5834598..."
1,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,car,39.109536,-94.580771,"{""type"":""Polygon"",""coordinates"":[[[-94.5807547..."
2,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,car,39.094360,-94.583396,"{""type"":""Polygon"",""coordinates"":[[[-94.5834062..."
3,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,car,39.095302,-94.583515,"{""type"":""Polygon"",""coordinates"":[[[-94.5835249..."
4,SENSITY-kc-9-CDDB22AD-31F6-4A11-BC94-CCC6602575A1,SENSITY-kc-9,2018-08-10 23:37:38.946,2018-08-10 23:39:01.415632,car,39.092984,-94.583621,"{""type"":""Polygon"",""coordinates"":[[[-94.5836287..."


In [4]:
df.shape

(162124, 8)

In [5]:
df.dropna(axis=0,how='any',inplace=True)
df.drop(columns='geometry',inplace=True)
df.drop(columns='session_type',inplace=True)

In [6]:
df.shape

(103943, 6)

In [7]:
df.rename(columns={'start_time':'start_date','end_time':'end_date'},inplace=True)

In [8]:
df.dtypes

uuid           object
asset_id       object
start_date     object
end_date       object
latitude      float64
longitude     float64
dtype: object

In [9]:
df['start_date'] = pd.to_datetime(df.start_date,format='%Y/%m/%d')
df['end_date'] = pd.to_datetime(df.end_date,format='%Y/%m/%d')

In [10]:
df.dtypes

uuid                  object
asset_id              object
start_date    datetime64[ns]
end_date      datetime64[ns]
latitude             float64
longitude            float64
dtype: object

In [11]:
df['start_from_fc']= df.start_date.dt.floor('5min')
df['end_from_fc'] = df.end_date.dt.ceil('5min')

In [12]:
df.isnull().sum()

uuid             0
asset_id         0
start_date       0
end_date         0
latitude         0
longitude        0
start_from_fc    0
end_from_fc      0
dtype: int64

In [13]:
df.dtypes

uuid                     object
asset_id                 object
start_date       datetime64[ns]
end_date         datetime64[ns]
latitude                float64
longitude               float64
start_from_fc    datetime64[ns]
end_from_fc      datetime64[ns]
dtype: object

In [14]:
n = (df.end_from_fc - df.start_from_fc)
n,(n.dt.total_seconds())/60
df['diff'] = (n.dt.total_seconds())/60

In [15]:
df.head(50)

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff
0,SENSITY-kc-23-40D98DC1-41F8-40EE-BBB8-E44DE41C...,SENSITY-kc-23,2018-08-10 23:33:26.081,2018-08-10 23:33:55.937100,39.109786,-94.583482,2018-08-10 23:30:00,2018-08-10 23:35:00,5.0
1,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,39.109536,-94.580771,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0
2,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,39.094360,-94.583396,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0
3,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,39.095302,-94.583515,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0
4,SENSITY-kc-9-CDDB22AD-31F6-4A11-BC94-CCC6602575A1,SENSITY-kc-9,2018-08-10 23:37:38.946,2018-08-10 23:39:01.415632,39.092984,-94.583621,2018-08-10 23:35:00,2018-08-10 23:40:00,5.0
5,SENSITY-kc-14-EC9EB215-CFAC-437F-9FEE-CF8917FF...,SENSITY-kc-14,2018-08-10 23:34:12.119,2018-08-10 23:39:38.118686,39.097486,-94.583381,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0
6,SENSITY-kc-18-B6079A6A-B9EF-4080-B4D8-B2BA7099...,SENSITY-kc-18,2018-08-10 23:36:28.757,2018-08-10 23:40:19.352096,39.108123,-94.583603,2018-08-10 23:35:00,2018-08-10 23:45:00,10.0
7,SENSITY-kc-2-2EB5BA3B-6629-4298-9D02-D292CE41641E,SENSITY-kc-2,2018-08-10 23:33:24.283,2018-08-10 23:47:47.458890,39.089526,-94.583630,2018-08-10 23:30:00,2018-08-10 23:50:00,20.0
8,SENSITY-kc-12-012FEE39-6092-419F-B096-31FA7A37...,SENSITY-kc-12,2018-08-10 23:47:25.020,2018-08-10 23:47:50.050454,39.095549,-94.583495,2018-08-10 23:45:00,2018-08-10 23:50:00,5.0
9,SENSITY-kc-21-8A06DAEC-A8E9-4996-8723-51570636...,SENSITY-kc-21,2018-08-10 23:47:51.283,2018-08-10 23:49:26.480149,39.109129,-94.580507,2018-08-10 23:45:00,2018-08-10 23:50:00,5.0


In [16]:
df[df['diff'].isnull()]

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff


In [17]:
df['slots'] = df['diff']/5
df['slots']=df['slots'].astype('int64')
df['bins'] = df.slots.map( lambda x: np.arange(1,x+1,1) if x > 1.0 else np.arange(1,x+1))

In [18]:
df.head()

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff,slots,bins
0,SENSITY-kc-23-40D98DC1-41F8-40EE-BBB8-E44DE41C...,SENSITY-kc-23,2018-08-10 23:33:26.081,2018-08-10 23:33:55.937100,39.109786,-94.583482,2018-08-10 23:30:00,2018-08-10 23:35:00,5.0,1,[1]
1,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,39.109536,-94.580771,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]"
2,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,39.094360,-94.583396,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]"
3,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,39.095302,-94.583515,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]"
4,SENSITY-kc-9-CDDB22AD-31F6-4A11-BC94-CCC6602575A1,SENSITY-kc-9,2018-08-10 23:37:38.946,2018-08-10 23:39:01.415632,39.092984,-94.583621,2018-08-10 23:35:00,2018-08-10 23:40:00,5.0,1,[1]


In [19]:
df.dtypes

uuid                     object
asset_id                 object
start_date       datetime64[ns]
end_date         datetime64[ns]
latitude                float64
longitude               float64
start_from_fc    datetime64[ns]
end_from_fc      datetime64[ns]
diff                    float64
slots                     int64
bins                     object
dtype: object

In [20]:
df['start'] = df.start_date.dt.date
df['start'] = pd.to_datetime(df.start,format='%Y\%m\%d')

In [21]:
df.head(5)

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff,slots,bins,start
0,SENSITY-kc-23-40D98DC1-41F8-40EE-BBB8-E44DE41C...,SENSITY-kc-23,2018-08-10 23:33:26.081,2018-08-10 23:33:55.937100,39.109786,-94.583482,2018-08-10 23:30:00,2018-08-10 23:35:00,5.0,1,[1],2018-08-10
1,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,39.109536,-94.580771,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]",2018-08-10
2,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,39.094360,-94.583396,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]",2018-08-10
3,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,39.095302,-94.583515,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,"[1, 2]",2018-08-10
4,SENSITY-kc-9-CDDB22AD-31F6-4A11-BC94-CCC6602575A1,SENSITY-kc-9,2018-08-10 23:37:38.946,2018-08-10 23:39:01.415632,39.092984,-94.583621,2018-08-10 23:35:00,2018-08-10 23:40:00,5.0,1,[1],2018-08-10


In [22]:
def explode(df, lst_cols, fill_value=''):
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    idx_cols = df.columns.difference(lst_cols)
    lens = df[lst_cols[0]].str.len()
    if (lens > 0).all():
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [23]:
new_df=explode(df, ['bins'], fill_value='')

In [24]:
new_df.isnull().sum()

uuid             0
asset_id         0
start_date       0
end_date         0
latitude         0
longitude        0
start_from_fc    0
end_from_fc      0
diff             0
slots            0
bins             0
start            0
dtype: int64

In [25]:
new_df[new_df['slots'].isnull()]

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff,slots,bins,start


In [26]:
new_df['bins'] = (new_df.start_from_fc + pd.to_timedelta(5*(new_df['bins']), unit='m'))
new_df['bins1'] = new_df.bins - pd.to_timedelta(5, unit='m')

In [27]:
new_df.head(100)

,uuid,asset_id,start_date,end_date,latitude,longitude,start_from_fc,end_from_fc,diff,slots,bins,start,bins1
0,SENSITY-kc-23-40D98DC1-41F8-40EE-BBB8-E44DE41C...,SENSITY-kc-23,2018-08-10 23:33:26.081,2018-08-10 23:33:55.937100,39.109786,-94.583482,2018-08-10 23:30:00,2018-08-10 23:35:00,5.0,1,2018-08-10 23:35:00,2018-08-10,2018-08-10 23:30:00
1,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,39.109536,-94.580771,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:35:00,2018-08-10,2018-08-10 23:30:00
2,SENSITY-kc-22-E5EBBB3A-05E4-4D9C-97CB-26542918...,SENSITY-kc-22,2018-08-10 23:34:58.454,2018-08-10 23:36:08.256733,39.109536,-94.580771,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:40:00,2018-08-10,2018-08-10 23:35:00
3,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,39.094360,-94.583396,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:35:00,2018-08-10,2018-08-10 23:30:00
4,SENSITY-kc-10-C4D472BA-65CA-4D2B-A6C0-9C3981CD...,SENSITY-kc-10,2018-08-10 23:33:40.424,2018-08-10 23:36:25.890125,39.094360,-94.583396,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:40:00,2018-08-10,2018-08-10 23:35:00
5,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,39.095302,-94.583515,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:35:00,2018-08-10,2018-08-10 23:30:00
6,SENSITY-kc-12-4C26D1EC-4CCE-4E9A-87D1-D93149F5...,SENSITY-kc-12,2018-08-10 23:34:44.149,2018-08-10 23:38:53.829785,39.095302,-94.583515,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:40:00,2018-08-10,2018-08-10 23:35:00
7,SENSITY-kc-9-CDDB22AD-31F6-4A11-BC94-CCC6602575A1,SENSITY-kc-9,2018-08-10 23:37:38.946,2018-08-10 23:39:01.415632,39.092984,-94.583621,2018-08-10 23:35:00,2018-08-10 23:40:00,5.0,1,2018-08-10 23:40:00,2018-08-10,2018-08-10 23:35:00
8,SENSITY-kc-14-EC9EB215-CFAC-437F-9FEE-CF8917FF...,SENSITY-kc-14,2018-08-10 23:34:12.119,2018-08-10 23:39:38.118686,39.097486,-94.583381,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:35:00,2018-08-10,2018-08-10 23:30:00
9,SENSITY-kc-14-EC9EB215-CFAC-437F-9FEE-CF8917FF...,SENSITY-kc-14,2018-08-10 23:34:12.119,2018-08-10 23:39:38.118686,39.097486,-94.583381,2018-08-10 23:30:00,2018-08-10 23:40:00,10.0,2,2018-08-10 23:40:00,2018-08-10,2018-08-10 23:35:00


In [29]:
 new_df.groupby(['bins1','bins','asset_id','latitude','longitude']).count()[['uuid']]

uuid
bins1               bins                asset_id      latitude  longitude       
2018-08-10 23:30:00 2018-08-10 23:35:00 SENSITY-kc-10 39.094261 -94.583400     1
                                                      39.094309 -94.583398     1
                                                      39.094360 -94.583396     1
                                        SENSITY-kc-12 39.095037 -94.583519     1
                                                      39.095089 -94.583516     1
                                                      39.095128 -94.583503     1
                                                      39.095233 -94.583507     1
                                                      39.095302 -94.583515     1
                                        SENSITY-kc-14 39.097431 -94.583387     1
                                                      39.097486 -94.583381     1
                                                      39.097544 -94.583376     1
                                                      39.097599 -94.583374     1
                                        SENSITY-kc-18 39.108078 -94.583837     1
                                                                -94.583837     1
                                                      39.108096 -94.583756     1
                                                                -94.583757     1
                                        SENSITY-kc-2  39.089526 -94.583630     1
                                                      39.089589 -94.583625     1
                                                      39.089637 -94.583625     1
                                        SENSITY-kc-22 39.109536 -94.580771     1
                                                      39.109590 -94.580789     1
                                        SENSITY-kc-23 39.109696 -94.583907     1
                                                      39.109711 -94.583836     1
                                                      39.109723 -94.583773     1
                                                      39.109740 -94.583698     1
                                                      39.109755 -94.583627     1
                                                      39.109771 -94.583553     1
                                                      39.109786 -94.583482     1
                                                      39.109868 -94.583097     1
                                        SENSITY-kc-8  39.092489 -94.583487     1
...                                                                          ...
2018-11-26 15:35:00 2018-11-26 15:40:00 SENSITY-kc-6  39.091312 -94.583540     1
                                        SENSITY-kc-9  39.092449 -94.583658     1
2018-11-26 15:40:00 2018-11-26 15:45:00 SENSITY-kc-11 39.093936 -94.583578     1
                                        SENSITY-kc-12 39.095432 -94.583501     1
                                        SENSITY-kc-14 39.098027 -94.583342     1
                                                      39.098083 -94.583335     1
                                        SENSITY-kc-15 39.100888 -94.583408     1
                                        SENSITY-kc-2  39.089601 -94.583638     1
                                        SENSITY-kc-6  39.091253 -94.583543     1
                                                      39.091312 -94.583540     1
2018-11-26 15:45:00 2018-11-26 15:50:00 SENSITY-kc-12 39.095432 -94.583501     1
                                        SENSITY-kc-15 39.100888 -94.583408     1
                                        SENSITY-kc-2  39.089651 -94.583639     1
                                        SENSITY-kc-5  39.090333 -94.584096     1
                                        SENSITY-kc-6  39.091253 -94.583543     1
                                                      39.091312 -94.583540     1
                                                      39.091470 -94.583535     1
                          

In [30]:
df_5min = new_df.groupby(['bins1','bins','asset_id','latitude','longitude']).count()[['uuid']].add_suffix('_Count').reset_index()

In [31]:
df_5min

,bins1,bins,asset_id,latitude,longitude,uuid_Count
0,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094261,-94.583400,1
1,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094309,-94.583398,1
2,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094360,-94.583396,1
3,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095037,-94.583519,1
4,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095089,-94.583516,1
5,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095128,-94.583503,1
6,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095233,-94.583507,1
7,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095302,-94.583515,1
8,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-14,39.097431,-94.583387,1
9,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-14,39.097486,-94.583381,1


In [32]:
df_5min.rename(columns={'bins1':'start_time','bins':'end_time','asset_id':'sensor_id'
                        ,'uuid_Count':'no_of_cars'}
               ,inplace=True)

In [34]:
df_5min.head()

,start_time,end_time,sensor_id,latitude,longitude,no_of_cars
0,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094261,-94.583400,1
1,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094309,-94.583398,1
2,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-10,39.094360,-94.583396,1
3,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095037,-94.583519,1
4,2018-08-10 23:30:00,2018-08-10 23:35:00,SENSITY-kc-12,39.095089,-94.583516,1


In [56]:
df_5min.loc[(df_5min['sensor_id']=='SENSITY-kc-7') & (df_5min['start_time']=='2018-08-23 13:30:00')].\
apply(lambda y: list([y['longitude'],y['latitude']]),axis=1).tolist()

[[-94.5837065388669, 39.0913863848405],
 [-94.58370732155409, 39.0913883653924],
 [-94.58369439450159, 39.091488652895]]

In [60]:
import folium
kanton_map = folium.Map(location=[39.0903, -94.58404627591267],
                   tiles='cartodbpositron', zoom_start=11)
kanton_map
kanton_map.choropleth(geo_data='KSBO.geojson')
kanton_map

In [61]:
import geopandas as gpd
dfj = gpd.read_file('KSBO.geojson')
print(dfj.columns)
dfj.head()

Index(['shape_area', 'nbhname', 'objectid', 'nbhid', 'shape_len', 'geometry'], dtype='object')


,shape_area,nbhname,objectid,nbhid,shape_len,geometry
0,14264150.3454,SENSITY-kc-14,68,113,21686.6610301,POLYGON ((-94.58333849174771 39.09799401202621...
1,14264150.3454,SENSITY-kc-12,68,113,21686.6610301,"POLYGON ((-94.58351310948109 39.0950265477875,..."
2,14264150.3454,SENSITY-kc-7,68,113,21686.6610301,"POLYGON ((-94.58370653886691 39.0913863848405,..."


In [84]:
kanton_map2 = folium.Map(location=[39.0903, -94.58404627591267], 
                    zoom_start=7.5)
kanton_map2.choropleth(geo_data='KSBO.geojson', data=del_df,
             columns=['sensor_id', 'no_of_cars_Count'],
             key_on='feature.properties.nbhname',
#              threshold_scale=[10, 20, 30],
             fill_color='BuPu')
kanton_map2

In [71]:
del_df=df_5min.loc[df_5min.sensor_id.isin(['SENSITY-kc-7','SENSITY-kc-12','SENSITY-kc-14']) & \
           (df_5min['start_time']=='2018-11-3 13:30:00')]

In [76]:
del_df= del_df.groupby(['start_time','end_time','sensor_id']).count()[['no_of_cars']].add_suffix('_Count').reset_index()

In [77]:
del_df

,start_time,end_time,sensor_id,no_of_cars_Count
0,2018-11-03 13:30:00,2018-11-03 13:35:00,SENSITY-kc-12,25
1,2018-11-03 13:30:00,2018-11-03 13:35:00,SENSITY-kc-14,131
2,2018-11-03 13:30:00,2018-11-03 13:35:00,SENSITY-kc-7,1
